# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
print(geoapify_key)

10c5e07d983f4092b8d7bde7e79c4cf2


In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Starter_Code/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,15.46,83,14,6.26,NZ,1713926596
1,1,puerto natales,-51.7236,-72.4875,4.07,89,100,2.49,CL,1713926312
2,2,hadibu,12.6500,54.0333,27.49,73,4,1.16,YE,1713926597
3,3,krasnozerskoye,53.9828,79.2374,-2.02,51,76,7.40,RU,1713926597
4,4,ribeira grande,38.5167,-28.7000,15.99,82,100,6.71,PT,1713926489


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,khasab,26.1799,56.2477,23.96,66,0,1.24,OM,1713926601
177,177,saint-pierre,-21.3393,55.4781,23.82,69,0,2.57,RE,1713926524
189,189,lhuentse,27.6679,91.1839,21.12,48,0,1.56,BT,1713926610
194,194,ramos arizpe,25.5500,-100.9667,23.86,46,0,3.09,MX,1713926610
256,256,gatesville,31.4352,-97.7439,21.38,89,0,3.60,US,1713926615
284,284,stephenville,32.2207,-98.2023,22.44,78,0,3.09,US,1713926458
347,347,dingzhou,38.5131,114.9956,22.06,17,0,2.76,CN,1713926621
368,368,al kharijah,25.4514,30.5464,25.56,18,0,2.19,EG,1713926623
395,395,al muwayh,22.4333,41.7583,22.23,18,0,4.23,SA,1713926625
454,454,ixtapa,20.7000,-105.2000,25.13,88,0,0.89,MX,1713926629


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] =""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
59,khasab,OM,26.1799,56.2477,66,
177,saint-pierre,RE,-21.3393,55.4781,69,
189,lhuentse,BT,27.6679,91.1839,48,
194,ramos arizpe,MX,25.5500,-100.9667,46,
256,gatesville,US,31.4352,-97.7439,89,
284,stephenville,US,32.2207,-98.2023,78,
347,dingzhou,CN,38.5131,114.9956,17,
368,al kharijah,EG,25.4514,30.5464,18,
395,al muwayh,SA,22.4333,41.7583,18,
454,ixtapa,MX,20.7000,-105.2000,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
    "apiKey":geoapify_key
}
   


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
khasab - nearest hotel: "Extra Divers Villa"
saint-pierre - nearest hotel: Tropic Hotel
lhuentse - nearest hotel: Hotel Shangrila
ramos arizpe - nearest hotel: Quinta Royal
gatesville - nearest hotel: Main Street Inn
stephenville - nearest hotel: No hotel found
dingzhou - nearest hotel: 大世界酒店
al kharijah - nearest hotel: Kharga Hotel
al muwayh - nearest hotel: فندق ومطعم الموية
ixtapa - nearest hotel: Ma. Cristina
sao joao da barra - nearest hotel: Pousada Mediterrâneo
aqaba - nearest hotel: Nairoukh Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
59,khasab,OM,26.1799,56.2477,66,"""Extra Divers Villa"""
177,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
189,lhuentse,BT,27.6679,91.1839,48,Hotel Shangrila
194,ramos arizpe,MX,25.5500,-100.9667,46,Quinta Royal
256,gatesville,US,31.4352,-97.7439,89,Main Street Inn
284,stephenville,US,32.2207,-98.2023,78,No hotel found
347,dingzhou,CN,38.5131,114.9956,17,大世界酒店
368,al kharijah,EG,25.4514,30.5464,18,Kharga Hotel
395,al muwayh,SA,22.4333,41.7583,18,فندق ومطعم الموية
454,ixtapa,MX,20.7000,-105.2000,88,Ma. Cristina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

    
# Display the map
hotel_plot 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)